In [1]:
# # test d'integration de plaidml. si tout marche bien, generaliser au training
# import plaidml.keras
# plaidml.keras.install_backend()
import os
import glob
import numpy as np
import pandas as pd
import radio
from radio.batchflow import FilesIndex, Dataset, Pipeline
from radio import CTImagesMaskedBatch as CTIMB
from radio_utils import show_slices, get_nodules_pixel_coords, num_of_cancerous_pixels
from radio.pipelines import split_dump

from config import config
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [32, 32]

# import utils
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

/home/paul/.local/lib/python3.6/site-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
PATH_TO_FOLDER = config['luna_raw']
cancerous_folder, non_cancerous_folder = '../jfr_pp/test/positive32', '../jfr_pp/test/negative32'

cancer_index = FilesIndex(path=os.path.join(cancerous_folder, '*'), dirs=True)
non_cancer_index = FilesIndex(path=os.path.join(non_cancerous_folder, '*'), dirs=True)
cancer_set = Dataset(cancer_index, batch_class=CTIMB)
non_cancer_set = Dataset(non_cancer_index, batch_class=CTIMB)

from radio.pipelines import combine_crops
crops_sampling = combine_crops(cancer_set, non_cancer_set, batch_sizes=(5, 5))

In [6]:
from radio.models import Keras3DUNet
from radio.models.keras.losses import dice_loss, tversky_loss

unet_config = dict(
    input_shape = (1, 32, 64, 64),
    num_targets = 1,
    loss= tversky_loss
)

In [7]:
from radio.batchflow import F

train_unet_pipeline = (
    combine_crops(cancer_set, non_cancer_set, batch_sizes=(4, 6))
    .init_model(
        name='3dunet', model_class=Keras3DUNet,
        config=unet_config, mode='static'
    )
    .train_model(
        name='3dunet',
        x=F(CTIMB.unpack, component='images', data_format='channels_first'),
        y=F(CTIMB.unpack, component='segmentation_targets', data_format='channels_first')
    )
)

In [8]:
%%time

N_ITERS = 1020

train_unet_pipeline.run()

# for i in range (N_ITERS):
#     train_unet_pipeline.next_batch(1)

keras_unet = train_unet_pipeline.get_model_by_name('3dunet')
keras_unet.save('./model/test.h5')

CPU times: user 36d 10h 25min 7s, sys: 2d 12h 6min 30s, total: 38d 22h 31min 38s
Wall time: 7d 7h 47min 48s
